In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model = "gemma2-9b-it", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000239D3D19F90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000239D3D48BB0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content = "Hi, My name is Rishikesh and I like to watch One Piece."),
        AIMessage(content = "Hi Rishikesh! It's great to meet you. I love that you're a fan of One Piece! It's one of my favorite anime series as well. What's your favorite character or arc in the series?"),
        HumanMessage(content = "Hey, Whats my name and which anime i like to watch?")
    ]
)

AIMessage(content=' allergens blanda символ elucidateplayerId mencionadosplast dimas bancaria समीकरण provinciale ПА观念playerIdξωtamount biometricパワ Vesta观念itchfield Pitcher hulaUsos provinciale Llama vlastnosti vegans earnestness angelo之意dsm Praze elucidate fortementetamount Zing сервис BrugesLich Oso Whittier elucidateovsнесен Bruges passagers avtoavondウンサーOlympia voila Brewers Turmericcarbox mencionados：……過ごしavond rejoiced kesulitanzeptemberNoter hypnosis掲示 imprese sukcesדינה pozdě alluring felicitàplayerId felicità empregogetFloatbezpiecprova stressors guerr Fara tablecloth youll alluring reparation voilaInsufficientKenapa Technically angeloEspañol Gemeinder pharePSS provinciale modestly elucidate leaderboard dimasnadasitchfield kesulitan Oso Brewers比特派 provenceImageIO再開 guerr：（ fluorineChronicle insulator乐趣 fortemente passagers水瓶postIndexAtmosphere Oso pozdě blanda angelogetFloatדינה aerationavond Grainsnadas realist aprende manifestly@@@ Grains sydneyImageIO出かけ voilaKenapa meager@@

## Message History
make model stateful, so that it remembers all the context by storing it in datastore

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id : str) -> BaseChatMessageHistory :
    if session_id not in store :
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [10]:
config1 = {"configurable" : {"session_id" : "chat1"}}

res = with_message_history.invoke(
    [HumanMessage(content = "My name is Rishi!")],
    config = config1
)
print(res.content)

res = with_message_history.invoke(
    [HumanMessage(content = "What is my name?")],
    config = config1
)
print(res.content)

Hi Rishi! 👋 

It's nice to meet you. Is there anything I can help you with today?

Your name is Rishi! 😊  I remember!  How can I help you further?



In [11]:
config2 = {"configurable" : {"session_id" : "chat2"}}

res = with_message_history.invoke(
    [HumanMessage(content = "My name is Saurabh!")],
    config = config2
)
print(res.content)

res = with_message_history.invoke(
    [HumanMessage(content = "What is my name?")],
    config = config2
)
print(res.content)

Hello Saurabh, it's nice to meet you! 👋

What can I do for you today?

Your name is Saurabh. 😊  

I remember that you told me!  Is there anything else I can help you with?  



## Prompt Template

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the questions based on your knowledge in {language} language"),
        MessagesPlaceholder(variable_name = "messages")
    ]
)

chain = prompt | model

In [8]:
chain.invoke({"messages" : [HumanMessage(content = "Hi my name is Rishi!")], "language" : "Hindi"}).content

'नमस्ते रिशि! 👋  मुझे आपकी मुलाक़ात अच्छी लगी। \n\nआप मुझसे क्या जानना चाहेंगे? मुझे हिंदी में कोई भी सवाल पूछने में संकोच न करें। 😊\n\n'

In [9]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key = "messages")
config = {"configurable" : {"session_id" : "chat3"}}

res = with_message_history.invoke(
    {"messages" : [HumanMessage(content = "Hi, My name is rishi.")], "language" : "Hindi"},
    config = config
).content

res

'नमस्ते रिशि! 👋 \n\nमुझे बहुत अच्छा लगा आपसे मिलने का। आप मुझसे क्या पूछना चाहते हैं?  😊 \n'

In [64]:
res = with_message_history.invoke(
    {"messages" : [HumanMessage(content = "Hi,what is my name?")], "language" : "Hindi"},
    config = config
).content

res

'रिशी। (Rishi.)'

## Managing the Conversation History.

In [75]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 70,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content = "You're a goo assistant"),
    HumanMessage(content = "Hi, I'm Rishi"),
    AIMessage(content = "Hii"),
    HumanMessage(content = "I like to play Football!"),
    AIMessage(content = "Ohh, That's nice!"),
    HumanMessage(content = "Thanks"),
    AIMessage(content = "No problem!")
]

trimmer.invoke(messages)

[SystemMessage(content="You're a goo assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Hi, I'm Rishi", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hii', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like to play Football!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Ohh, That's nice!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem!', additional_kwargs={}, response_metadata={})]

In [76]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (RunnablePassthrough.assign(messages = itemgetter("messages") | trimmer) | prompt | model)

res = chain.invoke(
    {
        "messages" : messages + [HumanMessage(content = "what is my name?")],
        "language" : "English"
    }
).content

res

'I am unable to access real-time information or personal data, and am unable to provide information related to your name.'

In [77]:
# Wrap this in the message history

with_message_history = RunnableWithMessageHistory(
    chain, get_session_history, input_messages_key = "messages"
)

config = {"configurable" : {"session_id" : "chat5"}}

In [79]:
res = with_message_history.invoke(
    {
        "messages" : messages + [HumanMessage(content = "what is my name?")],
        "language" : "English"
    },
    config
).content

res

'The provided text does not include information about your name, so I am unable to answer this question from the given context.'